In [1]:
!git clone https://github.com/eduardoccouto/ige_analytics.git

Cloning into 'ige_analytics'...
remote: Enumerating objects: 7, done.
remote: Counting objects: 100% (7/7), done.
remote: Compressing objects: 100% (7/7), done.
remote: Total 7 (delta 2), reused 2 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (7/7), 4.11 MiB | 5.85 MiB/s, done.
Resolving deltas: 100% (2/2), done.


In [2]:
import numpy as np


In [3]:
import pandas as pd

df = pd.read_csv("/content/ige_analytics/ibge_censo2022.csv", delimiter=',')

df.head()

,id_municipio,cor_raca,sexo,grupo_idade,alfabetizacao,populacao
0,1100023,Indígena,Homens,15 a 19 anos,Não alfabetizadas,NaN
1,1100262,Amarela,Mulheres,15 a 19 anos,Não alfabetizadas,NaN
2,1101005,Amarela,Mulheres,15 a 19 anos,Não alfabetizadas,NaN
3,1101435,Amarela,Mulheres,15 a 19 anos,Não alfabetizadas,NaN
4,1101435,Indígena,Mulheres,15 a 19 anos,Não alfabetizadas,NaN


Primeira etapa: Limpeza dos dados

In [4]:
df.isnull().sum()

,0
id_municipio,0
cor_raca,0
sexo,0
grupo_idade,0
alfabetizacao,0
populacao,250085


In [5]:
df['populacao'] = df['populacao'].fillna(0)
df

,id_municipio,cor_raca,sexo,grupo_idade,alfabetizacao,populacao
0,1100023,Indígena,Homens,15 a 19 anos,Não alfabetizadas,0.0
1,1100262,Amarela,Mulheres,15 a 19 anos,Não alfabetizadas,0.0
2,1101005,Amarela,Mulheres,15 a 19 anos,Não alfabetizadas,0.0
3,1101435,Amarela,Mulheres,15 a 19 anos,Não alfabetizadas,0.0
4,1101435,Indígena,Mulheres,15 a 19 anos,Não alfabetizadas,0.0
...,...,...,...,...,...,...
779795,2924009,Parda,Homens,65 anos ou mais,Alfabetizadas,1791.0
779796,4318903,Branca,Mulheres,65 anos ou mais,Alfabetizadas,2047.0
779797,4127700,Branca,Homens,65 anos ou mais,Alfabetizadas,4351.0
779798,3302700,Branca,Mulheres,65 anos ou mais,Alfabetizadas,7167.0


Segunda etapa: cálculo das taxas de variação

In [6]:
alfabetizacao_geral = df.groupby(['alfabetizacao', 'sexo', 'cor_raca'])['populacao'].sum().reset_index(name="Total").sort_values(by='Total', ascending=False)

alfabetizacao_geral

,alfabetizacao,sexo,cor_raca,Total
6,Alfabetizadas,Mulheres,Branca,36543564.0
8,Alfabetizadas,Mulheres,Parda,34177975.0
3,Alfabetizadas,Homens,Parda,31775413.0
1,Alfabetizadas,Homens,Branca,31687364.0
4,Alfabetizadas,Homens,Preta,8044704.0
9,Alfabetizadas,Mulheres,Preta,7882437.0
13,Não alfabetizadas,Homens,Parda,3331628.0
18,Não alfabetizadas,Mulheres,Parda,3029999.0
16,Não alfabetizadas,Mulheres,Branca,1594593.0
11,Não alfabetizadas,Homens,Branca,1498398.0


In [7]:
mulheres = alfabetizacao_geral[alfabetizacao_geral['sexo'] == 'Mulheres']
m = mulheres.pivot(index = 'alfabetizacao', columns='cor_raca', values='Total')

m.loc['Total'] = m.sum(axis=0)

m.loc['Taxa de alfabetização (%)'] = ((m.loc['Alfabetizadas'] / m.loc['Total'])*100).round(2)

m


cor_raca,Amarela,Branca,Indígena,Parda,Preta
alfabetizacao,,,,,
Alfabetizadas,392008.00,36543564.00,346820.00,34177975.00,7882437.00
Não alfabetizadas,10136.00,1594593.00,72148.00,3029999.00,831118.00
Total,402144.00,38138157.00,418968.00,37207974.00,8713555.00
Taxa de alfabetização (%),97.48,95.82,82.78,91.86,90.46


In [8]:
homens = alfabetizacao_geral[alfabetizacao_geral['sexo'] == 'Homens']
m = homens.pivot(index = 'alfabetizacao', columns='cor_raca', values='Total')

m.loc['Total'] = m.sum(axis=0)

m.loc['Taxa de alfabetização (%)'] = ((m.loc['Alfabetizadas'] / m.loc['Total'])*100).round(2)
m

cor_raca,Amarela,Branca,Indígena,Parda,Preta
alfabetizacao,,,,,
Alfabetizadas,343117.00,31687364.00,345363.00,31775413.00,8044704.00
Não alfabetizadas,8665.00,1498398.00,60303.00,3331628.00,966261.00
Total,351782.00,33185762.00,405666.00,35107041.00,9010965.00
Taxa de alfabetização (%),97.54,95.48,85.13,90.51,89.28
